In [13]:
import neurokit2 as nk
import configparser
import pickle
import os
import os.path as osp
from collections import defaultdict
from typing import List
from tqdm import tqdm

# Get WESAD CHEST dataset folder path

In [2]:
# Get WESAD Chest folder path
config_path = osp.join(osp.dirname(os.getcwd()), 'config.ini')
parser = configparser.ConfigParser()
parser.read(config_path)
wesad_dataset_path = parser['DATA_PATH']['wesad_dataset_path'] # Get WESAD dataset path first
wesad_chest_dataset_path = osp.join(wesad_dataset_path, 'wesad_chest_dataset.pkl')

In [3]:
# Load WESAD Chest dataset
wesad_chest = pickle.load(open(wesad_chest_dataset_path, 'rb'))

# Resampling WESAD-CHEST EDA signal

In [10]:
def resampling_data_signal(data: List[float], sampling_rate: int, desired_sampling_rate: int, method: str = 'interpolation') -> List[float]:
    output = nk.signal_resample(data, method = method, sampling_rate = sampling_rate, desired_sampling_rate = desired_sampling_rate)
    return output

In [11]:
# Sampling-rate of WESAD CHEST dataset recorded from clinical device is 700Hz
WESAD_CHEST_SAMPLING_RATE = 700 
# Sampling-rate of the desired resampling WESAD CHEST
WESAD_RESAMPLING_RATE = 4

In [20]:
# Start resampling signal data
eda_data = wesad_chest['eda']
eda_resampling = defaultdict(dict) 
resampling_ground_truth = defaultdict(dict)
for user_id, data in tqdm(eda_data.items()):
    for task_id, eda in data.items():
        resampled_eda = resampling_data_signal(eda, sampling_rate = WESAD_CHEST_SAMPLING_RATE, desired_sampling_rate = WESAD_RESAMPLING_RATE)
        eda_resampling[user_id][task_id] = resampled_eda
        resampling_ground_truth[user_id][task_id] = [list(set(wesad_chest['ground_truth'][user_id][task_id]))[0]] * len(resampled_eda)

100%|██████████| 15/15 [00:02<00:00,  6.46it/s]


In [21]:
# Create WESAD Resampling dataset
wesad_resampling = {
    'eda': eda_resampling,
    'ground_truth': resampling_ground_truth,
}

In [22]:
# Save the dataset
output_file_path = osp.join(wesad_dataset_path, 'wesad_chest_resampling_dataset.pkl')
pickle.dump(wesad_resampling, open(output_file_path, 'wb'))